In [2]:
library(hh.analytics)
library(hh.snowflake)

load.packages()

odbc        RJDBC         httr     jsonlite googlesheets    tidyverse 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
   lubridate     magrittr        tidyr        dplyr      stringr        purrr 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
       furrr      ggplot2       ggpubr      cowplot        rJava        mailR 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
    devtools     roxygen2 
        TRUE         TRUE

In [3]:
x = list.files() %>% 
    
    ##  Get Files Names
    data.frame(
        file_name = .
    ) %>% 
    filter(
        file_name %>% str_detect("\\.sql$") &
        (
            file_name %>% str_detect("^transform_(?:businessrules|criteria)") |
            file_name %>% str_detect("^transform_prediction_pool")
#             file_name %>% str_detect("^master_prediction_pool")
        )
    ) %>% 

    ##  Set Refresh Order
    mutate(
        priority = case_when(
            file_name %>% str_detect("^transform_businessrules")                     ~ 1,
            file_name %>% str_detect("^transform_criteria_client_allowed_contacts")  ~ 3,
            file_name %>% str_detect("^transform_criteria_client_allowed_")          ~ 2,
            file_name %>% str_detect("^transform_criteria")                          ~ 4,
            file_name %>% str_detect("^transform_prediction_pool")                   ~ 5,
            file_name %>% str_detect("^master_prediction_pool")                      ~ 6
        )
    ) %>% 
    arrange(
        priority,
        file_name
    ) %>% 

#     filter(
#         file_name %>% str_detect("^transform_prediction_pool")  |
#         file_name %>% str_detect("^transform_prediction_pool")  |
#         file_name %>% str_detect("^master_prediction_pool")
#     ) %>% 

    mutate(
        query = file_name %>% map_chr(.f = ~ .x %>% readLines() %>% paste0(collapse = "\n")),
        execute = query %>% map(.f = ~ .x %>% querySnowflake())
    )

Warning message in readLines(.):
"incomplete final line found on 'transform_businessrules_contact_cooldown.sql'"
Warning message in readLines(.):
"incomplete final line found on 'transform_businessrules_debtor_balance.sql'"
Warning message in readLines(.):
"incomplete final line found on 'transform_businessrules_debtor_experian.sql'"
Warning message in readLines(.):
"incomplete final line found on 'transform_businessrules_debtor_income.sql'"
Warning message in readLines(.):
"incomplete final line found on 'transform_businessrules_debtor_lastpayment.sql'"
Warning message in readLines(.):
"incomplete final line found on 'transform_businessrules_debtor_maturity.sql'"
Warning message in readLines(.):
"incomplete final line found on 'transform_businessrules_debtor_payplan.sql'"
Warning message in readLines(.):
"incomplete final line found on 'transform_businessrules_debtor_taxyear.sql'"
Warning message in readLines(.):
"incomplete final line found on 'transform_criteria_client_allowed_calls

In [4]:
x %>% unnest(execute) %>% group_by(status) %>% count()

status,n
<chr>,<int>
Table TRANSFORM_BUSINESSRULES_CONTACT_COOLDOWN successfully created.,1
Table TRANSFORM_BUSINESSRULES_DEBTOR_BALANCE successfully created.,1
Table TRANSFORM_BUSINESSRULES_DEBTOR_EXPERIAN successfully created.,1
Table TRANSFORM_BUSINESSRULES_DEBTOR_INCOME successfully created.,1
Table TRANSFORM_BUSINESSRULES_DEBTOR_LASTPAYMENT successfully created.,1
Table TRANSFORM_BUSINESSRULES_DEBTOR_MATURITY successfully created.,1
Table TRANSFORM_BUSINESSRULES_DEBTOR_PAYPLAN successfully created.,1
Table TRANSFORM_BUSINESSRULES_DEBTOR_TAXYEAR successfully created.,1
Table TRANSFORM_CRITERIA_ADDRESS_ALLOWED_EMAIL successfully created.,1
